# 1 数据预处理

In [9]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

对数据进行分词、编码

In [10]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def tokenize_function(examples):
    return tokenizer(examples["text"])

In [11]:
tokenized_dataset = datasets.map(tokenize_function, batched=True, num_proc=1, remove_columns="text")

对数据进行合并再切分，这样可以不是用padding来填充，有效利用空闲空间。

In [12]:
block_size = 128

def group_texts(examples):
    # 拼接所有文本
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # 我们将余数对应的部分去掉。但如果模型支持的话，可以添加padding，您可以根据需要定制此部件。
    total_length = (total_length // block_size) * block_size
    # 通过max_len进行分割。
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [13]:
lm_dataset = tokenized_dataset.map(group_texts, batched=True, batch_size=1000)

# 2 加载模型

In [14]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# 3 训练

In [21]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir= "output",
    eval_strategy= "epoch",
    learning_rate=2e-5,
    eval_delay=0.01
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"]
)

trainer.train()

C:\Users\chenzhen\anaconda3\envs\pytorch\Lib\site-packages\transformers\models\gpt2\modeling_gpt2.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch,Training Loss,Validation Loss
1,3.760600,3.666432
2,3.648700,3.647044
3,3.603700,3.643931


TrainOutput(global_step=7002, training_loss=3.693270469862745, metrics={'train_runtime': 873.737, 'train_samples_per_second': 64.094, 'train_steps_per_second': 8.014, 'total_flos': 1829109916237824.0, 'train_loss': 3.693270469862745, 'epoch': 3.0})

# 4 检验效果

In [52]:
# 微调以后的效果
model = AutoModelForCausalLM.from_pretrained("models/causallm-distilgpt2")
text = "Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input).logits
tokenizer.decode(output[0].argmax(-1))

' , the 2010 , the , and was the first game in the seriesalkyria Chronicles to Iting a same name system the elements tactical @-@ time combat , the predecessor , the game is'

In [53]:
# 原本模型输出效果
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
text = "Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input).logits
tokenizer.decode(output[0].argmax(-1))

' The the,. the. the was the first time in the series-ria Chronicles to Iting a same name system the and tactical-klife style,, the predecessor, the game is'